In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/Colab Notebooks/COSE474/FinalProject/20242R0136COSE47402/FinalProject

/content/drive/MyDrive/Colab Notebooks/COSE474/FinalProject/20242R0136COSE47402/FinalProject


In [3]:
ls

sample_image.png  TestCode.ipynb


In [4]:
dir = "/content/drive/MyDrive/Colab Notebooks/COSE474/FinalProject/20242R0136COSE47402/FinalProject/"

In [5]:
from PIL import Image
import requests
from transformers import CLIPProcessor, CLIPModel, VisionEncoderDecoderModel, AutoTokenizer

# Load the CLIP model and processor
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")

# Load the image captioning model
caption_model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
caption_tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.10k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/599M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/241 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

In [6]:
import torch

# load image
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

# Use CLIP to compute similarity scores for predefined captions
candidate_captions = ["a photo of a cat", "a photo of a dog", "a photo of an animal"]
inputs = clip_processor(text=candidate_captions, images=image, return_tensors="pt", padding=True)
outputs = clip_model(**inputs)

# Get the similarity scores
logits_per_image = outputs.logits_per_image
probs = logits_per_image.softmax(dim=1)
best_caption_index = torch.argmax(probs).item()
print(f"Best caption by CLIP: {candidate_captions[best_caption_index]}")


# Generate a detailed caption using the captioning model
pixel_values = clip_processor(images=image, return_tensors="pt").pixel_values
attention_mask = torch.ones(pixel_values.shape[:2], dtype=torch.long)

generated_ids = caption_model.generate(
    pixel_values,
    attention_mask=attention_mask,  # Explicitly pass the attention mask
    max_length=50,                 # Set a custom maximum length for the output
    pad_token_id=caption_tokenizer.pad_token_id  # Use the tokenizer's pad token ID
)
# generated_ids = caption_model.generate(pixel_values)

generated_caption = caption_tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print(f"Generated Caption: {generated_caption}")

Best caption by CLIP: a photo of a cat


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


Generated Caption: a cat laying on a blanket with a cat on it 


In [15]:
clip_model

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPSdpaAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e

In [16]:
for name, module in clip_model.named_modules():
  print(name, module)

 CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPSdpaAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1

In [17]:
ls

sample_image.png  TestCode.ipynb


In [24]:
self_attn_layer1 = clip_model.vision_model.encoder.layers[0].self_attn
print(self_attn_layer1)

# k_proj와 q_proj 확인
#print("Key Projection:", self_attn_layer1.k_proj)
#print("Query Projection:", self_attn_layer1.q_proj.weight)

# def hook(module, input, output):
#     return output
# self_attn_layer1.register_forward_hook(hook)

# Forward 함수 확인
import inspect
forward_code = inspect.getsource(self_attn_layer1.forward)
# print(forward_code)

# 파일에 저장
# file_name = "self_attention_layer1_forward.py"
# with open(file_name, "w") as f:
#     f.write(forward_code)

# print(f"Forward code saved to {file_name}")

CLIPSdpaAttention(
  (k_proj): Linear(in_features=768, out_features=768, bias=True)
  (v_proj): Linear(in_features=768, out_features=768, bias=True)
  (q_proj): Linear(in_features=768, out_features=768, bias=True)
  (out_proj): Linear(in_features=768, out_features=768, bias=True)
)


In [50]:
from typing import Optional, Tuple
from types import MethodType
from packaging import version
is_torch_greater_or_equal_than_2_2 = version.parse(torch.__version__) >= version.parse("2.2.0")


def forward_modified(
    self,
    hidden_states: torch.Tensor,
    attention_mask: Optional[torch.Tensor] = None,
    causal_attention_mask: Optional[torch.Tensor] = None,
    output_attentions: Optional[bool] = False,
) -> Tuple[torch.Tensor, Optional[torch.Tensor]]:
    if output_attentions:
        # TODO: Improve this warning with e.g. `model.config.attn_implementation = "manual"` once this is implemented.
        logger.warning_once(
            "CLIPModel is using CLIPSdpaAttention, but `torch.nn.functional.scaled_dot_product_attention` does not "
            "support `output_attentions=True`. Falling back to the manual attention implementation, but specifying "
            "the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can "
            'be removed using the argument `attn_implementation="eager"` when loading the model.'
        )
        return super().forward(
            hidden_states=hidden_states,
            attention_mask=attention_mask,
            causal_attention_mask=causal_attention_mask,
            output_attentions=output_attentions,
        )

    # CLIP text model uses both `causal_attention_mask` and `attention_mask`
    if attention_mask is not None and causal_attention_mask is not None:
        attn_mask = attention_mask + causal_attention_mask
    elif causal_attention_mask is not None:
        attn_mask = causal_attention_mask
    else:
        attn_mask = attention_mask

    bsz, tgt_len, embed_dim = hidden_states.size()

    query_states = self.q_proj(hidden_states)
    key_states = self.k_proj(hidden_states)
    value_states = self.v_proj(hidden_states)


    # print(query_states.size())
    # print(key_states.size())
    # print(value_states.size())


    query_states = query_states.view(bsz, -1, self.num_heads, self.head_dim).transpose(1, 2)
    key_states = key_states.view(bsz, -1, self.num_heads, self.head_dim).transpose(1, 2)
    value_states = value_states.view(bsz, -1, self.num_heads, self.head_dim).transpose(1, 2)

    # SDPA with memory-efficient backend is currently (torch==2.1.2) bugged with non-contiguous inputs with custom attn_mask,
    # Reference: https://github.com/pytorch/pytorch/issues/112577.
    if not is_torch_greater_or_equal_than_2_2 and query_states.device.type == "cuda" and attn_mask is not None:
        query_states = query_states.contiguous()
        key_states = key_states.contiguous()
        value_states = value_states.contiguous()

    # CLIP text model uses both `causal_attention_mask` and `attention_mask` sequentially.
    '''attn_output = torch.nn.functional.scaled_dot_product_attention(
        query_states,
        key_states,
        value_states,
        attn_mask=attn_mask,
        dropout_p=self.dropout if self.training else 0.0,
        scale=self.scale,
    )'''
    attn_output = value_states
    attn_output = attn_output.transpose(1, 2)
    attn_output = attn_output.reshape(bsz, tgt_len, embed_dim)

    attn_output = self.out_proj(attn_output)

    return attn_output, None

In [53]:
# 수정된 forward 함수를 Layer 1에 적용
# self_attn_layer1.forward = MethodType(forward_modified, self_attn_layer1)

for i in range(11):
  self_attn_layer_i = clip_model.vision_model.encoder.layers[i].self_attn
  self_attn_layer_i.forward = MethodType(forward_modified, self_attn_layer_i)

In [54]:
# Use CLIP to compute similarity scores for predefined captions
candidate_captions = ["a photo of a cat", "a photo of a dog", "a photo of an animal"]
inputs = clip_processor(text=candidate_captions, images=image, return_tensors="pt", padding=True)
outputs = clip_model(**inputs)

# Get the similarity scores
logits_per_image = outputs.logits_per_image
probs = logits_per_image.softmax(dim=1)
best_caption_index = torch.argmax(probs).item()
print(f"Best caption by CLIP: {candidate_captions[best_caption_index]}")

####################################################################################################

# Generate a detailed caption using the captioning model
pixel_values = clip_processor(images=image, return_tensors="pt").pixel_values
attention_mask = torch.ones(pixel_values.shape[:2], dtype=torch.long)

generated_ids = caption_model.generate(
    pixel_values,
    attention_mask=attention_mask,  # Explicitly pass the attention mask
    max_length=50,                 # Set a custom maximum length for the output
    pad_token_id=caption_tokenizer.pad_token_id  # Use the tokenizer's pad token ID
)
# generated_ids = caption_model.generate(pixel_values)

generated_caption = caption_tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print(f"Generated Caption: {generated_caption}")

Best caption by CLIP: a photo of an animal
Generated Caption: a cat laying on a blanket with a cat on it 


In [56]:
outputs.logits_per_image

tensor([[22.2181, 23.8571, 25.4977]], grad_fn=<TBackward0>)